# Synthetic Dataset

In [8]:
import os
import random
import string
import numpy as np
from PIL import Image, ImageDraw, ImageFont

# ==============================================================================
# CONFIGURACIÓN
# ==============================================================================
OUTPUT_DIR = "dataset_sintetico"
INTERSECTION_DIR = os.path.join(OUTPUT_DIR, "intersections")
NON_INTERSECTION_DIR = os.path.join(OUTPUT_DIR, "non_intersections")

os.makedirs(INTERSECTION_DIR, exist_ok=True)
os.makedirs(NON_INTERSECTION_DIR, exist_ok=True)

IMG_WIDTH = 256
IMG_HEIGHT = 256

N_IMAGES_INTERSECTION = 3000
N_IMAGES_NON_INTERSECTION = 3000

# ==============================================================================
# FUNCIONES AUXILIARES
# ==============================================================================
def add_gaussian_noise(image, mean=0, sigma=15):
    """
    Añade ruido Gaussiano a la imagen PIL.
    El parámetro sigma controla la fuerza del ruido.
    """
    img_array = np.array(image).astype(np.float32)
    noise = np.random.normal(mean, sigma, img_array.shape)
    noisy_img = img_array + noise
    noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_img)

def draw_bold_scaled_text(text):
    """
    Crea una imagen RGBA con el texto en “negrita” y de mayor tamaño.
    1. Dibuja el texto varias veces con offsets para simular negrita.
    2. Escala la imagen para duplicar el tamaño.
    Retorna la imagen RGBA resultante.
    """
    # Imagen temporal para el texto (un tamaño grande para no recortar el texto)
    temp_img = Image.new("RGBA", (200, 100), (0,0,0,0))
    temp_draw = ImageDraw.Draw(temp_img)
    
    # La fuente por defecto de PIL (no se puede cambiar tamaño real, 
    # pero podemos escalar la imagen después)
    font = ImageFont.load_default()
    
    # Para simular "negrita", dibujamos el mismo texto con un pequeño offset
    # en (0,0), (1,0), (0,1), (1,1), por ejemplo.
    # Escogemos la posición de inicio (5,5) para no recortar caracteres altos.
    base_x, base_y = 5, 5
    for dx in [0, 1]:
        for dy in [0, 1]:
            temp_draw.text((base_x + dx, base_y + dy), text, font=font, fill=(0,0,0,255))

    # Ahora medimos el bounding box real del texto (buscando píxeles no transparentes)
    # para recortar el exceso de área transparente.
    bbox = temp_img.getbbox()  # (left, top, right, bottom)
    if bbox is None:
        # Si el texto estuviera vacío, retorna la imagen tal cual
        cropped = temp_img
    else:
        cropped = temp_img.crop(bbox)
    
    # Escalamos la imagen al doble de tamaño, para que sea “letra grande”
    scaled_width = cropped.width * 2
    scaled_height = cropped.height * 2
    scaled = cropped.resize((scaled_width, scaled_height), Image.NEAREST)

    return scaled

def draw_rotated_text(base_image, text, x, y, angle):
    """
    Genera texto “negrita y grande”, lo rota y lo pega en (x, y) sobre base_image.
    """
    # Generamos la imagen con texto en negrita y escalado
    bold_text_img = draw_bold_scaled_text(text)
    
    # Rotamos con expand=True para no recortar
    rotated = bold_text_img.rotate(angle, expand=True)
    
    # Pegamos sobre base_image usando la propia imagen como máscara (canal alfa)
    rx, ry = rotated.size
    paste_box = (x, y, x + rx, y + ry)
    base_image.paste(rotated, paste_box, rotated)

def add_random_text(image):
    """
    Añade texto aleatorio (letras y dígitos) en posiciones y ángulos aleatorios,
    con “negrita y grande”.
    """
    n_texts = random.randint(1, 5)
    for _ in range(n_texts):
        # Contenido aleatorio
        text_length = random.randint(3, 8)
        text_content = ''.join(random.choices(string.ascii_letters + string.digits, k=text_length))
        
        # Posición aleatoria
        x_pos = random.randint(0, IMG_WIDTH - 50)
        y_pos = random.randint(0, IMG_HEIGHT - 50)
        
        # Ángulo aleatorio
        angle = random.randint(0, 359)
        
        draw_rotated_text(image, text_content, x_pos, y_pos, angle)

def create_street_line(center_x, center_y, angle_rad):
    """
    Crea los puntos (x1, y1, x2, y2) para una calle dada la posición (center_x, center_y)
    y un ángulo en radianes.
    """
    street_length = max(IMG_WIDTH, IMG_HEIGHT)
    dx = np.cos(angle_rad)
    dy = np.sin(angle_rad)
    x1 = center_x - street_length * dx
    y1 = center_y - street_length * dy
    x2 = center_x + street_length * dx
    y2 = center_y + street_length * dy
    return (x1, y1, x2, y2)

def draw_black_lines(draw, lines):
    """
    Recibe una lista de líneas (x1,y1,x2,y2) y las dibuja en negro
    con grosor aleatorio (15-25).
    """
    thickness_black = random.randint(15, 25)
    for (x1, y1, x2, y2) in lines:
        draw.line((x1, y1, x2, y2), fill="black", width=thickness_black)

def draw_white_lines(draw, lines):
    """
    Recibe la misma lista de líneas y las dibuja en blanco
    con grosor un poco menor (8-14).
    """
    thickness_white = random.randint(8, 14)
    for (x1, y1, x2, y2) in lines:
        draw.line((x1, y1, x2, y2), fill="white", width=thickness_white)

def draw_intersection(draw):
    """
    Dibuja 2 o 3 calles que se cruzan en un punto cerca del centro.
    1) Se pintan TODAS las líneas negras.
    2) Luego TODAS las líneas blancas.
    """
    center_x = random.randint(IMG_WIDTH // 3, 2 * IMG_WIDTH // 3)
    center_y = random.randint(IMG_HEIGHT // 3, 2 * IMG_HEIGHT // 3)

    # Puede cruzarse 2 o 3 calles
    n_streets = random.choice([2, 3])
    
    lines = []
    # Generamos n_streets ángulos y creamos cada línea
    base_angle = random.randint(0, 179)
    for i in range(n_streets):
        # Espaciar ángulos para que no sean todos muy cercanos
        offset = random.randint(60, 120) if i > 0 else 0
        angle = (base_angle + offset*(i)) % 180
        rad = np.radians(angle)
        line = create_street_line(center_x, center_y, rad)
        lines.append(line)
    
    # Primero TODAS las líneas en negro
    draw_black_lines(draw, lines)
    # Luego TODAS las líneas en blanco
    draw_white_lines(draw, lines)

def draw_random_non_intersection(draw):
    """
    En NO intersecciones NO puede haber líneas que se crucen.
    Por lo tanto, dibujaremos:
       - 1 sola calle (línea) en cualquier parte
       - o nada (espacio vacío).
    """
    choice = random.choice(["one_line", "nothing"])
    
    if choice == "nothing":
        return
    
    elif choice == "one_line":
        cx = random.randint(0, IMG_WIDTH)
        cy = random.randint(0, IMG_HEIGHT)
        angle = np.radians(random.randint(0, 179))
        line_seg = create_street_line(cx, cy, angle)
        
        # Pintar primero en negro, luego en blanco
        draw_black_lines(draw, [line_seg])
        draw_white_lines(draw, [line_seg])

def generate_dataset():
    # INTERSECTIONS
    print(f"Generando {N_IMAGES_INTERSECTION} imágenes de intersección...")
    for i in range(N_IMAGES_INTERSECTION):
        image = Image.new("RGB", (IMG_WIDTH, IMG_HEIGHT), color="white")
        draw = ImageDraw.Draw(image)
        
        # Dibuja 2 o 3 calles
        draw_intersection(draw)
        # Añade texto “negrita y grande” girado
        add_random_text(image)
        # Ruido Gaussiano
        image = add_gaussian_noise(image)
        
        fname = os.path.join(INTERSECTION_DIR, f"intersection_{i:04d}.png")
        image.save(fname)

    # NON-INTERSECTIONS
    print(f"Generando {N_IMAGES_NON_INTERSECTION} imágenes de NO intersección...")
    for i in range(N_IMAGES_NON_INTERSECTION):
        image = Image.new("RGB", (IMG_WIDTH, IMG_HEIGHT), color="white")
        draw = ImageDraw.Draw(image)
        
        # 0 o 1 línea que NO se cruce con otra
        draw_random_non_intersection(draw)
        # Añade texto “negrita y grande” girado
        add_random_text(image)
        # Ruido Gaussiano
        image = add_gaussian_noise(image)
        
        fname = os.path.join(NON_INTERSECTION_DIR, f"non_intersection_{i:04d}.png")
        image.save(fname)

    print("¡Dataset generado con éxito!")

if __name__ == "__main__":
    generate_dataset()


Generando 3000 imágenes de intersección...
Generando 3000 imágenes de NO intersección...
¡Dataset generado con éxito!


In [4]:
import torch
torch.cuda.is_available()

True

In [3]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt  

In [4]:
class BorderDetectionCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(BorderDetectionCNN, self).__init__()
        
        # Capas convolucionales
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # BatchNorm
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Dropout para regularización
        self.dropout = nn.Dropout(0.5)
        
        # MaxPool 2x2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Clasificador final
        # 224 -> 112 -> 56 -> 28 -> 14 (reducción cada pool)
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.pool(x)

        x = x.view(x.size(0), -1)  # Flatten

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [12]:
class BorderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.rotation_angles = [90, 180, 270]  # Ángulos para rotaciones adicionales
        
        borders_dir = os.path.join(root_dir, "borders")
        for img_name in os.listdir(borders_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
                img_path = os.path.join(borders_dir, img_name)
                self.samples.append((img_path, 0))
                for angle in self.rotation_angles:
                    self.samples.append((img_path, 0, angle))
        
        no_borders_dir = os.path.join(root_dir, "no_borders")
        for img_name in os.listdir(no_borders_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
                img_path = os.path.join(no_borders_dir, img_name)
                self.samples.append((img_path, 1))
                # for angle in self.rotation_angles:
                #     self.samples.append((img_path, 1, angle))
        


    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        if len(self.samples[idx]) == 2:
            # Imagen sin rotación
            img_path, label = self.samples[idx]
            image = Image.open(img_path).convert('RGB')
        else:
            # Imagen con rotación
            img_path, label, angle = self.samples[idx]
            image = Image.open(img_path).convert('RGB')
            image = image.rotate(angle)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def check_data_directory(data_dir):
    """Verifica que existan las carpetas necesarias con las imágenes"""
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"El directorio {data_dir} no existe")
    
    borders_dir = os.path.join(data_dir, "borders")
    no_borders_dir = os.path.join(data_dir, "no_borders")
    
    if not os.path.exists(borders_dir):
        raise FileNotFoundError(f"No se encuentra la carpeta 'borders' en {data_dir}")
    if not os.path.exists(no_borders_dir):
        raise FileNotFoundError(f"No se encuentra la carpeta 'no_borders' en {data_dir}")
    
    # Verificar que hay imágenes en las carpetas
    valid_extensions = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')
    borders_images = [f for f in os.listdir(borders_dir) if f.lower().endswith(valid_extensions)]
    no_borders_images = [f for f in os.listdir(no_borders_dir) if f.lower().endswith(valid_extensions)]
    
    if not borders_images:
        raise FileNotFoundError(f"No se encontraron imágenes válidas en {borders_dir}")
    if not no_borders_images:
        raise FileNotFoundError(f"No se encontraron imágenes válidas en {no_borders_dir}")
    
    print(f"Encontradas {len(borders_images)} imágenes con bordes")
    print(f"Después del data augmentation: {len(borders_images) * 4} imágenes con bordes")
    print(f"Encontradas {len(no_borders_images)} imágenes sin bordes")

In [14]:

def main():
    # ==============================================================================
    # CONFIGURACIÓN BÁSICA
    # ==============================================================================
    DATASET_DIR = "dataset_leo_synth"
    DATA_DIR = os.path.abspath(DATASET_DIR)
    BATCH_SIZE = 32
    EPOCHS = 10
    LEARNING_RATE = 1e-3
    
    # Verificar la estructura del directorio y las imágenes
    try:
        check_data_directory(DATA_DIR)
    except FileNotFoundError as e:
        print(f"Error al cargar los datos: {str(e)}")
        print("\nAsegúrate de que tu estructura de directorios sea así:")
        print(f"{DATASET_DIR}/")
        print("├── borders/")
        print("│   ├── imagen1.jpg")
        print("│   └── ...")
        print("└── no_borders/")
        print("    ├── imagen1.jpg")
        print("    └── ...")
        return

    # Dispositivo: GPU si está disponible, sino CPU
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Usando dispositivo: {DEVICE}")

    # ==============================================================================
    # TRANSFORMS Y DATASET
    # ==============================================================================
    data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
    ])

    # Usar nuestro dataset personalizado
    full_dataset = BorderDataset(DATA_DIR, transform=data_transforms)
    
    # Separar en train y valid (80% - 20%)
    total_size = len(full_dataset)
    train_size = int(0.8 * total_size)
    val_size = total_size - train_size

    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    # Ajustar num_workers a 0 para evitar bloqueos (sobre todo en Windows)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    print(f"Total de imágenes (incluyendo augmentation): {total_size}")
    print(f"Imágenes de entrenamiento: {train_size}")
    print(f"Imágenes de validación: {val_size}")

    # Instanciar el modelo y moverlo a GPU/CPU
    model = BorderDetectionCNN(num_classes=2).to(DEVICE)

    # Definir función de pérdida y optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    
    # Scheduler para ajuste dinámico del LR (opcional)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

    # ==============================================================================
    # ENTRENAMIENTO
    # ==============================================================================

    # -- Inicializar listas para graficar
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    

    def train_one_epoch(epoch):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            if (i + 1) % 10 == 0:
                print(f'  Época [{epoch+1}/{EPOCHS}], Batch [{i+1}/{len(train_loader)}], '
                      f'Loss: {loss.item():.4f}')
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100.0 * correct / total
        return epoch_loss, epoch_acc

    def validate_one_epoch():
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss = running_loss / len(val_loader)
        val_acc = 100.0 * correct / total
        return val_loss, val_acc

    best_val_loss = float('inf')
    start_time = time.time()
    
    for epoch in range(EPOCHS):
        print(f"==> Época {epoch+1}/{EPOCHS} <==")
        train_loss, train_acc = train_one_epoch(epoch)
        val_loss, val_acc = validate_one_epoch()
        
        # Guardar resultados
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        # Actualizar learning rate con el scheduler (si se usa)
        scheduler.step(val_loss)
        
        # Guardar el mejor modelo (según val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "mejor_modelo_bordes.pth")

        # Mostrar métricas de la época
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")
        print("-" * 60)

    total_time = time.time() - start_time
    print(f"Entrenamiento finalizado en {total_time:.2f} segundos.")
    print(f"Mejor modelo guardado como 'mejor_modelo_bordes.pth'")


if __name__ == "__main__":
    main()


Encontradas 1394 imágenes con bordes
Después del data augmentation: 5576 imágenes con bordes
Encontradas 5451 imágenes sin bordes
Usando dispositivo: cuda
Total de imágenes (incluyendo augmentation): 11027
Imágenes de entrenamiento: 8821
Imágenes de validación: 2206
==> Época 1/10 <==
  Época [1/10], Batch [10/276], Loss: 5.2493
  Época [1/10], Batch [20/276], Loss: 1.6225
  Época [1/10], Batch [30/276], Loss: 1.2976
  Época [1/10], Batch [40/276], Loss: 0.7775
  Época [1/10], Batch [50/276], Loss: 1.1146
  Época [1/10], Batch [60/276], Loss: 0.6917
  Época [1/10], Batch [70/276], Loss: 0.3678
  Época [1/10], Batch [80/276], Loss: 0.3132
  Época [1/10], Batch [90/276], Loss: 0.2705
  Época [1/10], Batch [100/276], Loss: 0.1488
  Época [1/10], Batch [110/276], Loss: 0.4389
  Época [1/10], Batch [120/276], Loss: 0.4443
  Época [1/10], Batch [130/276], Loss: 0.2338
  Época [1/10], Batch [140/276], Loss: 0.6017
  Época [1/10], Batch [150/276], Loss: 0.2675
  Época [1/10], Batch [160/276], 

In [9]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
import matplotlib.cm as cm
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms

# IMPORTANTE:
# Asegúrate de haber definido o importado tu modelo, por ejemplo:
#   from tu_archivo_modelo import BorderDetectionCNN
# Aquí se asume que lo tienes disponible en el entorno.

def generate_heatmap(
    image_path: str,
    model_path: str = "mejor_modelo_bordes.pth"
) -> np.ndarray:
    """
    Genera un heatmap de probabilidad de borde usando un modelo entrenado y un 
    enfoque de ventana deslizante. Devuelve el heatmap como matriz [0,1] y
    guarda la visualización superpuesta con sufijo '_heatmap.png'.

    Parámetros:
    -----------
    - image_path: Ruta de la imagen de entrada.
    - model_path: Ruta al modelo (checkpoint) entrenado.

    Retorna:
    --------
    - heatmap_normalized: np.ndarray con valores [0,1], mismo tamaño que la imagen original.
    """
    
    #--------------------------------------------------------------------------
    # 1. Cargar el modelo y moverlo a GPU/CPU
    #--------------------------------------------------------------------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BorderDetectionCNN(num_classes=2)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    #--------------------------------------------------------------------------
    # 2. Leer la imagen en color (RGB)
    #   * Sin convertir a escala de grises, para evitar discrepancias
    #--------------------------------------------------------------------------
    original_image = Image.open(image_path).convert("RGB")
    img_array = np.array(original_image)  # (alto, ancho, 3)
    height, width, _ = img_array.shape

    #--------------------------------------------------------------------------
    # 3. Definir tamaño y paso de la ventana deslizante
    #--------------------------------------------------------------------------
    WINDOW_SIZE = height // 75
    if WINDOW_SIZE < 2:
        WINDOW_SIZE = 2

    STEP_SIZE = WINDOW_SIZE // 2
    if STEP_SIZE < 1:
        STEP_SIZE = 1

    #--------------------------------------------------------------------------
    # 4. Inicializar heatmap y mapa de conteo
    #--------------------------------------------------------------------------
    heatmap = np.zeros((height, width), dtype=np.float32)
    count_map = np.zeros((height, width), dtype=np.float32)

    #--------------------------------------------------------------------------
    # 5. Transformaciones de entrada (mismas que en entrenamiento)
    #--------------------------------------------------------------------------
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std =[0.229, 0.224, 0.225]
        ),
    ])

    #--------------------------------------------------------------------------
    # 6. Recorrer la imagen con la ventana deslizante
    #--------------------------------------------------------------------------
    for y in range(0, height - WINDOW_SIZE, STEP_SIZE):
        for x in range(0, width - WINDOW_SIZE, STEP_SIZE):
            # Extraer el patch
            patch_3ch = img_array[y : y + WINDOW_SIZE, x : x + WINDOW_SIZE, :]

            # Transformar al tensor
            patch_tensor = transform(patch_3ch).unsqueeze(0).to(device)

            # Inferencia
            with torch.no_grad():
                logits = model(patch_tensor)
                probs = F.softmax(logits, dim=1)
            
            # Asumiendo que clase 0 = "no_borde" y clase 1 = "borde",
            # prob_border es la probabilidad de la clase 1
            prob_border = probs[0, 1].item()

            # Sumar la probabilidad en la región correspondiente
            heatmap[y : y + WINDOW_SIZE, x : x + WINDOW_SIZE] += prob_border
            count_map[y : y + WINDOW_SIZE, x : x + WINDOW_SIZE] += 1

    #--------------------------------------------------------------------------
    # 7. Normalizar el heatmap [0,1]
    #--------------------------------------------------------------------------
    count_map = np.maximum(count_map, 1e-5)  # evitar división por cero
    heatmap /= count_map  # promedio de las probabilidades
    h_min, h_max = heatmap.min(), heatmap.max()
    heatmap_normalized = (heatmap - h_min) / (h_max - h_min + 1e-8)

    #--------------------------------------------------------------------------
    # 8. Generar visualización superpuesta y guardarla
    #--------------------------------------------------------------------------
    # Convertir de RGB a BGR para usar cv2.addWeighted correctamente
    base_img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

    cmap = cm.get_cmap('jet')
    heatmap_color = cmap(heatmap_normalized)[..., :3]  # quitar canal alpha
    heatmap_color = (heatmap_color * 255).astype(np.uint8)

    alpha = 0.5  # transparencia
    overlay = cv2.addWeighted(base_img_bgr, 1.0 - alpha, heatmap_color, alpha, 0)

    # Guardar resultado
    base_name, _ = os.path.splitext(os.path.basename(image_path))
    output_filename = f"{base_name}_heatmap.png"
    cv2.imwrite(output_filename, overlay)
    print(f"Heatmap guardado en: {output_filename}")

    #--------------------------------------------------------------------------
    # 9. Retornar el heatmap normalizado como np.ndarray
    #--------------------------------------------------------------------------
    return heatmap_normalized

heatmap_mat = generate_heatmap("habana.jpg", "mejor_modelo_bordes.pth")

C:\Users\leona\AppData\Local\Temp\ipykernel_11936\2628937369.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Heatmap guardado en: habana_heatmap.png


In [6]:
import torch
import torch.nn.functional as F
from torch import nn

class GradCAM:
    """
    Implementación de Grad-CAM para PyTorch.
    
    Uso:
    ----
    1) Instanciar: gradcam = GradCAM(model, target_layer=model.conv4)
    2) Forward:    output = gradcam.forward(input_tensor)
    3) Backward:   output[:, class_idx].backward(retain_graph=True)
    4) Generar:    cam = gradcam.generate(class_idx=class_idx)
       ( 'cam' será una lista de mapas [batch_size] con valores [0,1] )
    """
    def __init__(self, model, target_layer):
        """
        :param model: El modelo (p.e. instancia de BorderDetectionCNN).
        :param target_layer: Capa (módulo) del modelo donde enganchar hooks.
                             E.g. model.conv4
        """
        self.model = model
        self.target_layer = target_layer
        
        # Aquí guardaremos la activación y el gradiente de esa capa
        self.activation = None
        self.gradient = None
        
        # Registrar hooks
        self._register_hooks()
    
    def _register_hooks(self):
        """
        Registra hooks para guardar la activación (forward) 
        y el gradiente (backward) de la capa objetivo.
        """
        def forward_hook(module, input, output):
            # Activación de la capa
            self.activation = output.detach()

        def backward_hook(module, grad_in, grad_out):
            # grad_out[0] es el gradiente con respecto a la salida de la capa
            self.gradient = grad_out[0].detach()

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)
    
    def forward(self, x):
        """
        Ejecuta forward en el modelo completo y devuelve la salida (logits).
        """
        return self.model(x)
    
    def generate(self, class_idx, eps=1e-8):
        """
        Genera el mapa de Grad-CAM para la clase 'class_idx' 
        usando la activación y gradiente guardados por los hooks.
        
        :param class_idx: Índice de la clase objetivo (p.e. 1 para "borde").
        :return: Lista de CAMs (np.array) normalizados [0,1], uno por 
                 cada elemento en el batch (forma [batch_size, h, w]).
        """
        # La forma de self.activation es [B, C, H, W]
        # La forma de self.gradient   es [B, C, H, W]
        
        # 1) Promediar gradiente espacialmente: alpha_k = mean(grad_k)
        #    (Promedio por cada canal k)
        alpha = self.gradient.view(self.gradient.size(0), 
                                   self.gradient.size(1), -1).mean(dim=2) 
        # alpha shape: [B, C]

        # 2) Ponderar la activación por alpha
        #    Expandir alpha para que sea [B, C, 1, 1]
        alpha = alpha.unsqueeze(-1).unsqueeze(-1)  
        weighted_activation = alpha * self.activation
        
        # 3) Sumar canales: CAM = ReLU( sum_k( alpha_k * A_k ) )
        cam = weighted_activation.sum(dim=1, keepdim=True)  # [B,1,H,W]
        cam = F.relu(cam)
        
        # 4) Normalizar cada CAM individualmente a [0,1]
        cams_result = []
        for i in range(cam.size(0)):
            # cam[i] -> [1, H, W]
            single_cam = cam[i, 0, :, :].cpu().numpy()
            min_v, max_v = single_cam.min(), single_cam.max()
            single_cam = (single_cam - min_v) / (max_v - min_v + eps)
            cams_result.append(single_cam)
        
        return cams_result


In [5]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
import matplotlib.cm as cm
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms

# Asegúrate de haber definido tu modelo (BorderDetectionCNN) y de tener el GradCAM definido
# from tu_archivo_modelo import BorderDetectionCNN
# from tu_archivo_gradcam import GradCAM

def generate_heatmap_gradcam(
    image_path: str,
    model_path: str = "mejor_modelo_bordes.pth"
) -> np.ndarray:
    """
    Genera un "heatmap" usando Grad-CAM por ventanas deslizantes.
    Se superpone cada CAM local al patch correspondiente en la imagen global.
    Finalmente, guarda y retorna el mapa normalizado (0,1).
    """

    #--------------------------------------------------------------------------
    # 1. Cargar el modelo y moverlo a GPU/CPU
    #--------------------------------------------------------------------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BorderDetectionCNN(num_classes=2)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    #--------------------------------------------------------------------------
    # 2. Definir Grad-CAM en la capa deseada (e.g. conv4)
    #--------------------------------------------------------------------------
    gradcam = GradCAM(model, target_layer=model.conv4)

    #--------------------------------------------------------------------------
    # 3. Leer la imagen en color (RGB)
    #--------------------------------------------------------------------------
    original_image = Image.open(image_path).convert("RGB")
    img_array = np.array(original_image)  # (alto, ancho, 3)
    height, width, _ = img_array.shape

    #--------------------------------------------------------------------------
    # 4. Definir tamaño y paso de la ventana deslizante
    #--------------------------------------------------------------------------
    WINDOW_SIZE = height // 75
    if WINDOW_SIZE < 2:
        WINDOW_SIZE = 2

    STEP_SIZE = WINDOW_SIZE // 2
    if STEP_SIZE < 1:
        STEP_SIZE = 1

    #--------------------------------------------------------------------------
    # 5. Inicializar un mapa para Grad-CAM y un mapa de conteo
    #--------------------------------------------------------------------------
    gradcam_map = np.zeros((height, width), dtype=np.float32)
    count_map = np.zeros((height, width), dtype=np.float32)

    #--------------------------------------------------------------------------
    # 6. Transformaciones de entrada (mismas que en entrenamiento)
    #--------------------------------------------------------------------------
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std =[0.229, 0.224, 0.225]
        ),
    ])

    #--------------------------------------------------------------------------
    # 7. Recorrer la imagen con la ventana deslizante
    #--------------------------------------------------------------------------
    for y in range(0, height - WINDOW_SIZE, STEP_SIZE):
        for x in range(0, width - WINDOW_SIZE, STEP_SIZE):
            # Extraer el patch original
            patch_3ch = img_array[y : y + WINDOW_SIZE, x : x + WINDOW_SIZE, :]

            # Transformar al tensor (224x224)
            patch_tensor = transform(patch_3ch).unsqueeze(0).to(device)
            
            #------------------------------------------------------------------
            # (a) Forward del modelo a través de Grad-CAM
            #------------------------------------------------------------------
            model.zero_grad()
            output = gradcam.forward(patch_tensor)  # logits -> [1,2]
            
            # Aquí elegimos la clase "borde" (índice 1) o podrías usar la clase
            # predicha: class_idx = output.argmax(dim=1).item()
            class_idx = 1

            #------------------------------------------------------------------
            # (b) Backward para Grad-CAM (solo de la clase_idx)
            #------------------------------------------------------------------
            target = output[:, class_idx]
            target.backward(retain_graph=True)
            
            #------------------------------------------------------------------
            # (c) Generar la cam normalizada [0,1]
            #------------------------------------------------------------------
            cam_list = gradcam.generate(class_idx=class_idx)
            cam_patch = cam_list[0]  # batch_size=1, tomamos el primero

            # cam_patch está en tamaño [feature_h, feature_w] (ej. 14x14 si
            # la salida de la última conv es 14x14), la habíamos ampliado
            # a 224 en la parte de `transform`? 
            #   -> Realmente NO, la red produce un feature map mas pequeño 
            #      (p.ej 14x14), y la transform es a la entrada no a la salida.
            #
            # Para superponer en la imagen "patch" (224x224) debemos:
            # 1) reescalar 'cam_patch' a (224,224) -> "cam_resized_224"
            # 2) luego, si queremos ponerlo en la ventana real (WINDOW_SIZE),
            #    reescalamos 'cam_resized_224' a (WINDOW_SIZE, WINDOW_SIZE).
            #
            # Haremos la versión "Grad-CAM local" con respecto al parche de 224x224
            # y luego lo reescalamos a la ventana real (WINDOW_SIZE, WINDOW_SIZE).
            
            cam_patch_torch = torch.from_numpy(cam_patch).unsqueeze(0).unsqueeze(0) 
            cam_resized_224 = F.interpolate(cam_patch_torch, size=(224,224), mode='bilinear', align_corners=False)
            cam_resized_224 = cam_resized_224.squeeze().cpu().numpy()  # shape (224,224)
            
            # Ahora reescalar de (224,224) a (WINDOW_SIZE, WINDOW_SIZE)
            cam_resized = cv2.resize(cam_resized_224, (WINDOW_SIZE, WINDOW_SIZE))
            
            #------------------------------------------------------------------
            # (d) Acumular en gradcam_map
            #------------------------------------------------------------------
            gradcam_map[y : y + WINDOW_SIZE, x : x + WINDOW_SIZE] += cam_resized
            count_map[y : y + WINDOW_SIZE, x : x + WINDOW_SIZE] += 1

    #--------------------------------------------------------------------------
    # 8. Normalizar gradcam_map [0,1]
    #--------------------------------------------------------------------------
    count_map = np.maximum(count_map, 1e-5) 
    gradcam_map /= count_map
    gc_min, gc_max = gradcam_map.min(), gradcam_map.max()
    gradcam_map_norm = (gradcam_map - gc_min) / (gc_max - gc_min + 1e-8)

    #--------------------------------------------------------------------------
    # 9. Generar visualización superpuesta y guardarla
    #--------------------------------------------------------------------------
    base_img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

    cmap = cm.get_cmap('jet')
    heatmap_color = cmap(gradcam_map_norm)[..., :3]  # quitar canal alpha
    heatmap_color = (heatmap_color * 255).astype(np.uint8)

    alpha = 0.5
    overlay = cv2.addWeighted(base_img_bgr, 1.0 - alpha, heatmap_color, alpha, 0)

    # Guardar resultado
    base_name, _ = os.path.splitext(os.path.basename(image_path))
    output_filename = f"{base_name}_gradcam.png"
    cv2.imwrite(output_filename, overlay)
    print(f"Grad-CAM guardado en: {output_filename}")

    return gradcam_map_norm


# Ejemplo de uso:
gradcam_map = generate_heatmap_gradcam("habana.jpg", "mejor_modelo_bordes.pth")


C:\Users\leona\AppData\Local\Temp\ipykernel_11936\3074028824.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

NameError: name 'GradCAM' is not defined

# DATASET ANOTATOR

In [7]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageDraw
import os

# Configuración inicial
input_folder = r"E:\Universidad\ML\ML-Project\data\Generales-Parciales"
output_folder = os.path.join(input_folder, "annotations")
os.makedirs(output_folder, exist_ok=True)

# Variables globales
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
current_index = 0
rect_start = None
rect_end = None
rect_id = None
zoom_level = 1.0
current_image = None
img_tk = None
offset_x = 0
offset_y = 0
pan_start = None

# Funciones principales
def load_image(index):
    if 0 <= index < len(image_files):
        image_path = os.path.join(input_folder, image_files[index])
        image = Image.open(image_path)
        return image
    return None

def save_rectangle(image, start, end, output_path):
    cropped = image.crop((min(start[0], end[0]), min(start[1], end[1]),
                          max(start[0], end[0]), max(start[1], end[1])))
    cropped.save(output_path)

def on_mouse_press(event):
    global rect_start, rect_id, pan_start
    if event.num == 1:  # Left click
        rect_start = (int((event.x - offset_x) / zoom_level), int((event.y - offset_y) / zoom_level))
        rect_id = canvas.create_rectangle(event.x, event.y, event.x, event.y, outline="red")
    elif event.num == 3:  # Right click
        pan_start = (event.x, event.y)

def on_mouse_drag(event):
    global rect_id, offset_x, offset_y, pan_start
    if rect_id and rect_start:
        canvas.coords(rect_id, rect_start[0] * zoom_level + offset_x, rect_start[1] * zoom_level + offset_y, event.x, event.y)
    elif pan_start:
        dx = event.x - pan_start[0]
        dy = event.y - pan_start[1]
        offset_x += dx
        offset_y += dy
        pan_start = (event.x, event.y)
        update_canvas()

def on_mouse_release(event):
    global rect_start, rect_end, rect_id, current_image, pan_start
    if event.num == 1 and rect_start:  # Left click release
        rect_end = (int((event.x - offset_x) / zoom_level), int((event.y - offset_y) / zoom_level))
        if rect_start and rect_end:
            output_path = os.path.join(output_folder, f"annotation_{current_index}_{rect_start[0]}_{rect_start[1]}_{rect_end[0]}_{rect_end[1]}.png")
            save_rectangle(current_image, rect_start, rect_end, output_path)
        rect_start = None
        rect_end = None
        rect_id = None
    elif event.num == 3:  # Right click release
        pan_start = None

def on_mouse_wheel(event):
    global zoom_level, offset_x, offset_y
    factor = 1.1 if event.delta > 0 else 0.9
    new_zoom_level = zoom_level * factor

    # Adjust offsets to zoom relative to the cursor position
    cursor_x = canvas.canvasx(event.x)
    cursor_y = canvas.canvasy(event.y)
    offset_x = cursor_x - factor * (cursor_x - offset_x)
    offset_y = cursor_y - factor * (cursor_y - offset_y)

    zoom_level = new_zoom_level
    update_canvas()

def next_image():
    global current_index, current_image, img_tk, zoom_level, offset_x, offset_y
    current_index += 1
    zoom_level = 1.0
    offset_x = 0
    offset_y = 0
    if current_index < len(image_files):
        current_image = load_image(current_index)
        update_canvas()
    else:
        print("No hay más imágenes.")

def update_canvas():
    global img_tk
    if current_image:
        resized_image = current_image.resize((int(current_image.width * zoom_level), int(current_image.height * zoom_level)))
        img_tk = ImageTk.PhotoImage(resized_image)
        canvas.delete("all")
        canvas.config(scrollregion=(0, 0, resized_image.width + offset_x, resized_image.height + offset_y))
        canvas.create_image(offset_x, offset_y, anchor=tk.NW, image=img_tk)

# Crear la interfaz gráfica
root = tk.Tk()
root.title("Herramienta de Anotación Rápida")

canvas = tk.Canvas(root)
canvas.pack(fill=tk.BOTH, expand=True)

frame_buttons = tk.Frame(root)
frame_buttons.pack()

btn_next = tk.Button(frame_buttons, text="Siguiente Imagen", command=next_image)
btn_next.pack(side=tk.LEFT)

canvas.bind("<ButtonPress-1>", on_mouse_press)
canvas.bind("<B1-Motion>", on_mouse_drag)
canvas.bind("<ButtonRelease-1>", on_mouse_release)
canvas.bind("<MouseWheel>", on_mouse_wheel)
canvas.bind("<ButtonPress-3>", on_mouse_press)
canvas.bind("<B3-Motion>", on_mouse_drag)
canvas.bind("<ButtonRelease-3>", on_mouse_release)

# Cargar la primera imagen
if image_files:
    current_image = load_image(current_index)
    update_canvas()
else:
    print("No se encontraron imágenes en la carpeta especificada.")

root.mainloop()


# Random sampler

In [23]:
import os
import random
from PIL import Image

# Configuración inicial
input_folder = r"E:\Universidad\ML\ML-Project\data\Generales-Parciales"
output_folder = os.path.join(input_folder, "random_crops")
os.makedirs(output_folder, exist_ok=True)

# Número total de subimágenes y tamaño del recorte
num_total_crops = 5600
crop_size = 200

# Obtener la lista de imágenes
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
num_images = len(image_files)
if num_images == 0:
    raise ValueError("No se encontraron imágenes en la carpeta especificada.")

# Cantidad de recortes por imagen
crops_per_image = num_total_crops // num_images

# Generar recortes aleatorios
def generate_random_crops(image_path, num_crops, crop_size, output_folder):
    with Image.open(image_path) as img:
        width, height = img.size
        for i in range(num_crops):
            if width < crop_size or height < crop_size:
                raise ValueError(f"La imagen {image_path} es más pequeña que el tamaño del recorte ({crop_size}x{crop_size}).")

            left = random.randint(0, width - crop_size)
            top = random.randint(0, height - crop_size)
            right = left + crop_size
            bottom = top + crop_size

            crop = img.crop((left, top, right, bottom))
            crop_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_crop_{i}.png"
            crop.save(os.path.join(output_folder, crop_filename))

# Procesar cada imagen
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    generate_random_crops(image_path, crops_per_image, crop_size, output_folder)

print(f"Se generaron {num_total_crops} recortes aleatorios de {crop_size}x{crop_size} y se guardaron en la carpeta {output_folder}.")


c:\Users\leona\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (128440320 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\leona\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (111917568 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\leona\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (106997760 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\leona\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (106093056 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\leona\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (106574208 pixels) exceeds limit of 89478485 pixels, could be dec

Se generaron 5600 recortes aleatorios de 200x200 y se guardaron en la carpeta E:\Universidad\ML\ML-Project\data\Generales-Parciales\random_crops.
